# Credit Risk Resampling Techniques

In [70]:
import warnings
warnings.filterwarnings('ignore')

In [71]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [72]:
# Load the data
file_path = Path('Data/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [73]:
# Create our features
X = pd.get_dummies(df, columns=["homeowner"])
X = X.drop(columns="loan_status")
#X.head()
# Create our target
y = df[['loan_status']]#.ravel()
#y.head()

In [74]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [75]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [76]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=78, 
                                                    stratify=y)
X_train.shape

(58152, 10)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [77]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [78]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [79]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [80]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [81]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9901346127192281

In [82]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  616,     9],
       [  100, 18659]], dtype=int64)

In [83]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred, digits=6))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk   0.860335  0.985600  0.994669  0.918717  0.990124  0.979457       625
   low_risk   0.999518  0.994669  0.985600  0.997088  0.990124  0.981235     18759

avg / total   0.995030  0.994377  0.985892  0.994561  0.990124  0.981178     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [105]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [106]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [108]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_nr = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_nr)

0.995174881390266

In [87]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_nr)

array([[  623,     2],
       [  121, 18638]], dtype=int64)

In [88]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_nr, digits=6))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk   0.837366  0.996800  0.993550  0.910153  0.995174  0.990692       625
   low_risk   0.999893  0.993550  0.996800  0.996711  0.995174  0.990049     18759

avg / total   0.994652  0.993655  0.996695  0.993920  0.995174  0.990069     19384



### SMOTE Oversampling

In [89]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

# View the count of target classes with Counter
from collections import Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [90]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [91]:
# Calculated the balanced accuracy score
y_pred_smote = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_smote)

0.995174881390266

In [92]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smote)

array([[  623,     2],
       [  121, 18638]], dtype=int64)

In [93]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote, digits=6))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk   0.837366  0.996800  0.993550  0.910153  0.995174  0.990692       625
   low_risk   0.999893  0.993550  0.996800  0.996711  0.995174  0.990049     18759

avg / total   0.994652  0.993655  0.996695  0.993920  0.995174  0.990069     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [109]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids 

X_resampled, y_resampled = ClusterCentroids(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

# View the count of target classes with Counter
from collections import Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [110]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [111]:
# Calculate the balanced accuracy score
y_pred_cc = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_cc)

0.9956813049736126

In [112]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_cc)

array([[  623,     2],
       [  102, 18657]], dtype=int64)

In [113]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc, digits=6))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk   0.859310  0.996800  0.994563  0.922963  0.995681  0.991602       625
   low_risk   0.999893  0.994563  0.996800  0.997221  0.995681  0.991158     18759

avg / total   0.995360  0.994635  0.996728  0.994826  0.995681  0.991173     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [114]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN 
X_resampled, y_resampled = SMOTEENN(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)
# View the count of target classes with Counter
from collections import Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [115]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [116]:
# Calculate the balanced accuracy score
y_pred_smteen = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_smteen)

0.9953081507543047

In [117]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smteen)

array([[  623,     2],
       [  116, 18643]], dtype=int64)

In [118]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smteen, digits=6))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk   0.843031  0.996800  0.993816  0.913490  0.995307  0.990932       625
   low_risk   0.999893  0.993816  0.996800  0.996845  0.995307  0.990341     18759

avg / total   0.994835  0.993913  0.996704  0.994158  0.995307  0.990360     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.


In [120]:
print(f"1. Under Sampling produced 0.9956813049736126 accuracy score")
print(f"2. Under Sampling produced 0.994635 recall")
print(f"3. Under Sampling had the best Geometric Mean Score 0.995681")

1. Under Sampling produced 0.9956813049736126 accuracy score
2. Under Sampling produced 0.994635 recall
3. Under Sampling had the best Geometric Mean Score 0.995681
